##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet                                                                                                                    # IGNORE_COPYRIGHT: cleared by OSS licensing
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Перенос обучения и тонкая настройка

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/transfer_learning"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/transfer_learning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/transfer_learning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/images/transfer_learning.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве вы узнаете, как классифицировать изображения кошек и собак с помощью передачи обучения из предварительно обученной модели.

Предварительно обученная модель - это сохраненная сеть, которая ранее была обучена на большом наборе данных, обычно на крупномасштабной задаче классификации изображений. Вы можете либо использовать предварительно обученную модель как есть, либо использовать перенос обучения, и настроить вашу модель для данной задачи.

Идея, лежащая в основе переноса обучения для классификации изображений, заключается в том, что если модель обучается на достаточно большом и достаточно общем наборе данных, то эта модель будет эффективно служить в качестве общей модели визуального мира. Таким образом вы можете воспользоваться преимуществами изученных структур признаков без необходимости начинать с нуля обучение большой модели на большом наборе данных.

В этом ноутбуке вы попробуете два способа использования предварительно обученной модели:
 
1. Извлечение признаков: используйте представления, полученные в предыдущей сети, для извлечения значимых признаков из новых образцов. Вы можете просто добавить новый классификатор, который будет обучаться с нуля, поверх предварительно обученной модели, чтобы вы могли перенастроить функции извлечения признаков, обученные ранее в перенесенной модели.

 Вам не нужно повторно тренировать всю модель. Базовая сверточная сеть уже содержит функции, которые обычно полезны для классификации изображений. Однако последняя часть предварительно обученной модели специфична для исходной задачи классификации, и как результат - для набора классов, на которых была обучена модель.

1. Тонкая настройка: разморозьте несколько верхних слоев основной замороженной модели и совместно тренируйте их и новые добавленные слои классификатора. Это позволяет нам более «тонко настроить» представления функций высокого порядка в базовой модели, чтобы сделать их более подходящими для конкретной задачи.

Вы будете следовать общему рабочему процессу машинного обучения.

1. Изучение и понимание данных
1. Создание конвейера ввода данных, в данном случае Keras ImageDataGenerator.
1. Создание модели.
   * Загрузка предварительно обученной базовой модели (и предварительно обученных весов)
   * Добавление сверху ваших собственных слоев классификации
1. Обучение модели
1. Оценка модели


In [ ]:
!pip install tf-nightly

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

## Предобработка данных

### Загрузка данных

В этом уроке вы будете использовать набор данных, содержащий несколько тысяч изображений кошек и собак. Загрузите и распакуйте zip-файл, содержащий изображения, затем создайте два tf.data.Dataset-а, для обучения и для проверки, с помощью утилиты tf.keras.preprocessing.image_dataset_from_directory. Вы можете узнать больше о загрузке изображений в этом [руководстве](https://www.tensorflow.org/tutorials/load_data/images).

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

In [ ]:
validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Покажем первые 9 изображений из тренировочного датасета:

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Поскольку исходный набор данных не содержит тестового датасета, вы создадите его. Для этого определите, сколько пакетов данных доступно в проверочном датасете, используя tf.data.experimental.cardinality, а затем переместите 20% из них в тестовый датасет.

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
print('Количество проверочных пакетов: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Количество тестовых пакетов: %d' % tf.data.experimental.cardinality(test_dataset))

### Конфигурирование датасета для повышения производительности

Используйте буферизованную предварительную выборку для загрузки изображений с диска без блокировки ввода-вывода. Чтобы узнать больше об этом методе, см. Руководство [производительность данных](https://www.tensorflow.org/guide/data_performance).

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Аугментация данных

Если у вас нет большого набора данных изображений, рекомендуется искусственно создавать различные реалистичные образцы, применяя случайные преобразования к исходным изображениям, например поворот или горизонтальное отражение. Это предоставит модели различные варианты обучающих данных и уменьшит [переобучение] (https://www.tensorflow.org/tutorials/keras/overfit_and_underfit). Вы можете узнать больше об аугментации данных в этом [руководстве](https://www.tensorflow.org/tutorials/images/data_augmentation).

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

Примечание: эти слои активны только в процессе тренировки, когда вы вызываете `model.fit`. Они неактивны, когда модель используется в режиме оценки - `model.evaulate` или прогнозирования - ` model.predict`.

Давайте несколько раз применим эти слои к одному и тому же изображению и посмотрим на результат.

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### Масштабирование значений пикселей

Сейчас вы загрузите `tf.keras.applications.MobileNetV2` для использования в качестве базовой модели. Эта модель ожидает значений пикселей в диапазоне `[-1,1]`, но на данный момент значения пикселей в ваших изображениях находятся в диапазоне `[0-255]`. Чтобы изменить их масштаб, используйте имеющийся в модели метод предварительной обработки `preprocess_input`.

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

Примечание. В качестве альтернативы вы можете изменять масштаб значений пикселей с `[0,255]` на `[-1, 1]`, используя слой [Rescaling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling).

In [ ]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

Примечание: при использовании других моделей из `tf.keras.applications` обязательно проверьте документацию API, чтобы определить, ожидают ли они пикселей в диапазоне `[-1,1]` или `[0,1]`, или используйте встроенную функцию `preprocess_input`.

## Создание базовой модели из предварительно обученной сверточной сети

Вы создадите базовую модель на основе модели **MobileNet V2**, разработанной в Google. Эта модель предварительно обучена на ImageNet, большом наборе данных, состоящем из 1,4 млн изображений и 1000 классов. ImageNet - это набор данных для исследовательской подготовки с широким спектром категорий, таких как `jackfruit` и `syringe`. Эта база знаний поможет нам классифицировать кошек и собак по нашему конкретному набору данных.

Во-первых, вам нужно выбрать, какой уровень MobileNet V2 вы будете использовать для извлечения признаков. Самый последний уровень классификации («верхний», поскольку большинство диаграмм моделей машинного обучения идут снизу вверх) не очень полезен. Вместо этого вы будете следовать общепринятой практике - полагаться на самый последний слой перед операцией flatten(разворачивание матрицы в одномерный массив). Этот слой называется «узким местом». Элементы слоя «узкое место» сохраняют большую универсальность по сравнению с последним/верхним слоем.

Сначала создайте экземпляр модели MobileNet V2 с предварительно загруженными весами, обученными на ImageNet. Указав аргумент **include_top = False**, вы загружаете сеть, которая не включает верхние слои классификации, что идеально подходит для извлечения функций.

In [ ]:
# Создание базовой модели из предварительно обученной модели MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Эта базовая модель преобразует входное изображение из матрицы `160x160x3` в матрицу `5x5x1280`. Давайте посмотрим, что он делает с примером пакета изображений:

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Извлечение признаков

На следующем шаге вы заморозите сверточную базу, созданную на предыдущем шаге, и будете использовать ее в качестве средства извлечения признаков. Кроме того, вы добавите сверху классификатор и обучите классификатор верхнего уровня.

### Заморозка светрочной сети

Перед компиляцией и обучением модели важно заморозить сверточную сеть. Замораживание происходит путем установки параметра layer.trainable = False и предотвращает обновление весов в данном слое во время обучения. MobileNet V2 имеет много слоев, поэтому установка флага для всей модели в значение False заморозит их все.

In [ ]:
base_model.trainable = False

### Важное примечание о слоях BatchNormalization

Многие модели содержат слои `tf.keras.layers.BatchNormalization`. Этот слой является особым случаем, и следует принимать определенные меры предосторожности в процессе тонкой настройки, как показано далее в этом руководстве.

Когда вы устанавливаете `layer.trainable = False`, слой `BatchNormalization` будет работать в режиме вывода и не будет обновлять свою статистику среднего значения и дисперсии.

Когда вы размораживаете модель, содержащую слои `BatchNormalization`, чтобы выполнить точную настройку, вы должны держать слои `BatchNormalization` в режиме вывода, передавая `training = False` при вызове базовой модели. В противном случае обновления, применяемые к необучаемым весам, уничтожат то, чему модель научилась.

Подробности см. В [Руководстве по переносу обучения](https://www.tensorflow.org/guide/keras/transfer_learning).

In [ ]:
# Давайте посмотрим на архитектуру модели
base_model.summary()

### Добавление верхних слоев классификации

Чтобы получить прогнозы из блока признаков, выполните усреднение по матрице `5x5`, используя слой `tf.keras.layers.GlobalAveragePooling2D` для преобразования матрицы признаков `5х5х960` в один вектор из 1280 элементов для каждого изображения.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Apply a `tf.keras.layers.Dense` layer to convert these features into a single prediction per image. You don't need an activation function here because this prediction will be treated as a `logit`, or a raw prediction value.  Positive numbers predict class 1, negative numbers predict class 0.

Примените слой `tf.keras.layers.Dense`, чтобы преобразовать эти признаки в один прогноз для изображения. Здесь вам не нужна функция активации, потому что этот прогноз будет рассматриваться как `logit` или необработанное значение прогноза. Положительные числа предсказывают класс 1, отрицательные числа предсказывают класс 0.

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

Постройте модель, объединив вместе слои аугментации, масштабирования, базовую модель и слои извлечения признаков с помощью [Keras Functional API](https://www.tensorflow.org/guide/keras/functional). Как упоминалось ранее, используйте `training = False` для базовой модели, поскольку наша модель содержит слои `BatchNormalization`.

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

### Компиляция модели

Скомпилируйте модель перед ее обучением. Поскольку существует два класса, используйте бинарную кросс-энтропию как фунцию потерь, с парамтром `from_logits = True`, поскольку модель предуматривает линейный вывод.

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

2.2 млн параметров в MobileNet заморожены, но в полносвязном Dense слое есть 1,2 тыс. _trainable_(тренируемых) параметров. Они распределены между двумя объектами `tf.Variable` - весами и смещениями.

In [ ]:
len(model.trainable_variables)

### Обучение модели

После обучения в течении 10 эпох, вы должны увидеть точность ~94% на проверочном датасете

In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print("Начальная величина потерь: {:.2f}".format(loss0))
print("Начальная точность: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

### График обучения

Давайте посмотрим на кривые точности/потери обучения и проверки при использовании базовой модели MobileNet V2 в качестве экстрактора фиксированных признаков.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Примечание. Если вам интересно, почему метрики валидации явно лучше, чем метрики обучения, главный фактор заключается в том, что уровни `tf.keras.layers.BatchNormalization` и `tf.keras.layers.Dropout` влияют на точность во время обучения, но отключаются при расчете величины потерь при валидации.

В меньшей степени это также связано с тем, что метрики обучения показывают среднее значение за эпоху, тогда как метрики валидации оцениваются после окончания эпохи, поэтому метрики валидации видят модель, которая тренировалась немного дольше.

## Тонкая настройка

В эксперименте по извлечению признаков вы тренировали только несколько слоев поверх базовой модели MobileNet V2. Веса предварительно обученной сети **не** обновлялись во время обучения.

Один из способов еще больше повысить точность - это обучить(или «тонко настроить») веса верхних слоев предварительно обученной модели вместе с обучением добавленного вами классификатора. В процессе обучения веса должны быть настроены с общих признаков на признаки, связанные с вашим набором данных.

Примечание. Это следует делать только после того, как вы обучили классификатор верхнего уровня с предварительно обученной и замороженной базовой моделью. Если вы добавите случайно инициализированный классификатор поверх предварительно обученной модели и попытаетесь обучить все слои вместе, величина обновлений градиента будет слишком большой (из-за случайных весов классификатора), и ваша предварительно обученная модель просто забудет то, чему она предварительно научилась.

Кроме того, вам следует попытаться настроить небольшое количество верхних уровней, а не всю модель MobileNet. В большинстве сверточных сетей чем выше уровень, тем он более специализирован. Первые несколько слоев изучают очень простые и общие функции, которые распространяются практически на все типы изображений. По мере того, как вы поднимаетесь выше, функции становятся все более специфичными для набора данных, на котором была обучена модель. Цель точной настройки - адаптировать эти специализированные функции для работы с новым набором данных, а не перезаписывать общее обучение.

### Разморозка верхних уровней модели


Все, что вам нужно сделать, это разморозить `base_model` и сделать нижние слои недоступными для обучения. Затем вам следует перекомпилировать модель (необходимо, чтобы эти изменения вступили в силу) и возобновить обучение.

In [ ]:
base_model.trainable = True

In [ ]:
# Посмотрим сколько слоев в базовой модели
print("Количество словев базовой модели: ", len(base_model.layers))

# Выполняем точную настройку, начиная с этого слоя.
fine_tune_at = 100

# Замораживаем все слои до `fine_tune_at`
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### Компиляция модели

Поскольку вы тренируете гораздо большую модель и хотите повторно адаптировать предварительно обученные веса, на этом этапе важно использовать более низкую скорость обучения. В противном случае ваша модель может очень быстро переобучиться. Поэтому уменьшим base_learning_rate в 10 раз

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Продолжение обучения модели

Если на этапе предыдущего обучения вы получили схождение, то этот шаг повысит вашу точность на несколько процентов. Заметьте, что обучение начинается с эпохи, взятой из истории предыдущего обучения.

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Давайте посмотрим на кривые точности/потери обучения и проверки при тонкой настройке последних нескольких уровней базовой модели MobileNet V2 и обучении классификатора поверх нее. 
Если величина потерь при валидации намного выше, чем при тренировке, значит ваша модель переобучилась.
Вы также можете столкнуться с переобучением, поскольку новый обучающий набор относительно невелик и похож на исходные наборы данных MobileNet V2.

После тонкой настройки модель получает почти 98% точности на проверочном датасете.

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Оценка и прогнозирование

Наконец, вы можете проверить точность модели на новых данных с помощью тестового датасета.

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Точность на тестовых данных :', accuracy)

Теперь все готово, чтобы использовать эту модель для предсказаний, будет ли ваш питомец кошкой или собакой.

In [ ]:
# Получаем пакет данных из тестового датасета
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Применяем сигмоиду, поскольку наша модель возвращает logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

## Заключение

* **Использование предварительно обученной модели для извлечения признаков**: при работе с небольшим набором данных обычной практикой является использование возможностей модели, обученной на более крупном наборе данных в том же домене. Это делается путем создания экземпляра предварительно обученной модели и добавления полносвязного классификатора(Dense) сверху. Предварительно обученная модель «заморожена», и во время обучения обновляются только веса классификатора.
В этом случае сверточная сеть извлекает все признаки, связанные с каждым изображением, и вы обучаете классификатор, который определяет класс изображения с учетом этого набора извлеченных признаков.

* **Тонкая настройка предварительно обученной модели**: для дальнейшего повышения производительности можно переназначить верхние уровни предварительно обученных моделей для нового набора данных с помощью тонкой настройки.
В этом случае вы настраиваете свои веса так, чтобы ваша модель изучала признаки в верхних слоях, в то время, как остальные слои остаются замороженные. Этот метод обычно рекомендуется, когда обучающий набор данных большой и очень похож на исходный набор данных, на котором была обучена предварительно обученная модель.

Чтобы узнать больше, посетите [Руководство по переносу обучения](https://www.tensorflow.org/guide/keras/transfer_learning).
